In [ ]:
import movingpeople as mp

from shapely import Point, Polygon
import geopandas as gpd
import osmnx as ox
import pandas as pd

In [ ]:
!pip show movingpeople

In [ ]:
help(mp.generate_routes)

In [ ]:
help(mp.visualise_route)

In [ ]:
# Search query for a geographic area
query = "City of Westminster"
# Get the walking network for the query location
G = ox.graph.graph_from_place(query, network_type="walk", simplify=True)
# Project the graph to WGS84
Gp = ox.project_graph(G, to_crs="4326")

In [ ]:
# To make randomised routes from a single origin
routes = mp.generate_routes(
                            Gp,
                            time_from="2020-02-26 20:42:53",
                            time_until="2020-02-26 21:42:53",
                            time_strategy="random",
                            route_strategy="one-many",
                            origin_destination_coords=[51.499127, -0.153522],
                            total_routes=10,
                            walk_speed=1.4,
                            frequency="1s",
                            )

In [ ]:
df = pd.DataFrame(
    {
        "Name": ["A", "B"],
        "Latitude": [51.50727, 51.50127],
        "Longitude": [-0.154522, -0.153922],
    }
)
gdf = gpd.GeoDataFrame(
    df, geometry=gpd.points_from_xy(df.Longitude, df.Latitude), crs="EPSG:4326"
)
area = gdf.to_crs(crs=3857) 
area['geometry'] = area['geometry'].buffer(250)

In [ ]:
clipped_routes = mp.clip_routes_to_polygon(routes, area)

In [ ]:
entry_exit = mp.get_entry_exit_times(clipped_routes)

In [ ]:
clipped_routes['time'] = clipped_routes['time'].astype(str)

In [ ]:
routes['time'] = routes['time'].astype(str)

In [ ]:
map_plot = mp.visualise_route(routes, 500)
map_plot.add_data(data=area, name='potential sites')
map_plot.add_data(data=clipped_routes, name='routes within site radius')

In [ ]:
map_plot.save_to_html(file_name='first_map.html')